In [ ]:
%%time
import os
import numpy as np
os.chdir('../src/')
from util import clean_data as cd
from util import viz
import algos.basic as basic
import algos.similarity as sim
METADATA_PATH = '../data/Phenotypic_V1_0b_preprocessed1.csv'
PREFIX = '../data/rois_ez/'
POSTFIX = '_rois_ez.1D'

In [ ]:
%%time
md = cd.get_metadata(METADATA_PATH, ['FILE_ID', 'DX_GROUP'])

In [ ]:
%%time
f_names = cd.get_filenames(md, PREFIX, POSTFIX)

In [ ]:
%%time
data = cd.get_data(f_names)

In [ ]:
%%time
data = cd.run_function(basic.transpose, data)
#data = [np.array([[1,2,3,4], [4,5,6,7]]),
#        np.array([[12,5,3,4], [4,9,6,7]])]

In [ ]:
%%time
data = cd.run_function(basic.mean_center, data)

In [ ]:
%%time
data = cd.run_function(sim.dtw, data)

In [ ]:
viz.heatmap_channels(data,
                     2,
                     title='Raw Data Heatmap',
                     xtitle='Timestep',
                     ytitle='ROI',
                     ztitle='4D Intensity (mean centered)')